# 문서 검색에서의 가설적 문서 임베딩 (HyDE)

## 개요

이 코드는 문서 검색을 위한 **가설적 문서 임베딩(Hypothetical Document Embedding, HyDE)** 시스템을 구현한 것입니다. HyDE는 쿼리 질문을 가설적 문서로 변환하여, 벡터 공간에서 쿼리와 문서 간의 분포 차이를 줄이는 혁신적인 접근 방식을 제공합니다. HyDE는 쿼리를 답을 포함한 가설적 문서로 확장함으로써 쿼리와 문서의 벡터 분포를 더욱 유사하게 만들어 검색의 정확도를 향상시키는 것을 목표로 합니다.

---

## 동기

전통적인 검색 방법은 짧은 쿼리와 길고 상세한 문서 사이의 의미적 차이(semantic gap)로 인해 어려움을 겪는 경우가 많습니다. HyDE는 쿼리를 가설적 문서로 확장하여 쿼리 표현을 문서 표현과 유사하게 만들어 이 문제를 해결합니다. 이를 통해 벡터 공간에서의 쿼리와 문서 간의 관련성을 향상시킬 수 있습니다.

---

## 주요 구성 요소

1. PDF 처리 및 텍스트 청크 분할
2. FAISS와 OpenAI 임베딩을 사용한 벡터 스토어 생성
3. 가설적 문서 생성을 위한 언어 모델(GPT-4)
4. HyDE 기법을 구현한 **HyDERetriever** 클래스

---

## 방법 세부 사항

### 문서 전처리 및 벡터 스토어 생성

1. PDF 문서를 처리하고 작은 청크로 분할합니다.
2. OpenAI 임베딩을 사용하여 FAISS 벡터 스토어를 생성하여 효율적인 유사성 검색을 수행합니다.

### 가설적 문서 생성

1. 언어 모델(GPT-4)을 사용하여 쿼리에 답변하는 가설적 문서를 생성합니다.
2. 생성은 벡터 스토어의 청크 크기와 일치하는, 상세한 가설적 문서가 되도록 프롬프트 템플릿을 통해 조정됩니다.

### 검색 프로세스

**HyDERetriever** 클래스는 다음과 같은 단계로 검색을 수행합니다:

1. 쿼리로부터 언어 모델을 사용하여 가설적 문서를 생성합니다.
2. 생성된 가설적 문서를 벡터 스토어의 검색 쿼리로 사용합니다.
3. 이 가설적 문서와 가장 유사한 문서들을 검색하여 반환합니다.

---

## 주요 특징

1. **쿼리 확장**: 짧은 쿼리를 상세한 가설적 문서로 변환하여 검색 정확도를 높입니다.
2. **유연한 구성**: 청크 크기, 중첩 범위, 검색할 문서 수를 조정할 수 있습니다.
3. **OpenAI 모델과의 통합**: GPT-4를 사용해 가설적 문서를 생성하고 OpenAI 임베딩을 통해 벡터 표현을 만듭니다.

---

## 이 접근법의 이점

1. **향상된 관련성**: 쿼리를 전체 문서로 확장함으로써 더 세부적이고 관련성 높은 검색 결과를 얻을 수 있습니다.
2. **복잡한 쿼리 처리**: 복잡하거나 다면적인 쿼리에 대해 보다 적합한 검색 결과를 제공하는 데 유리합니다.
3. **적응성**: 가설적 문서 생성은 다양한 유형의 쿼리 및 문서 도메인에 적응할 수 있습니다.
4. **문맥 이해 능력 향상**: 확장된 쿼리는 원래 질문의 의도와 문맥을 더 잘 파악할 수 있게 합니다.

---

## 구현 세부 사항

1. 가설적 문서 생성을 위해 OpenAI의 ChatGPT 모델을 사용합니다.
2. 벡터 공간에서 효율적인 유사성 검색을 위해 FAISS를 사용합니다.
3. 가설적 문서와 검색 결과를 쉽게 시각화할 수 있는 기능을 제공합니다.

---

## 결론

**가설적 문서 임베딩(HyDE)**은 쿼리와 문서 간의 의미적 차이를 해결하는 혁신적인 문서 검색 접근법입니다. 고급 언어 모델을 활용하여 쿼리를 가설적 문서로 확장함으로써, HyDE는 검색 관련성을 크게 향상시킬 가능성을 가지고 있습니다. 특히 법률 연구, 학술 문헌 검토 또는 고급 정보 검색 시스템과 같이 쿼리 의도와 문맥 이해가 중요한 분야에서 매우 유용하게 사용될 수 있습니다.


<div style="text-align: center;">

<img src="../images/HyDe.svg" alt="HyDe" style="width:40%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/hyde-advantages.svg" alt="HyDe" style="width:100%; height:auto;">
</div>

### Import libraries

In [2]:
import os
import sys
from dotenv import load_dotenv


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document(s) path

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### Define the HyDe retriever class - creating vector store, generating hypothetical document, and retrieving

In [4]:
class HyDERetriever:
    def __init__(self, files_path, chunk_size=500, chunk_overlap=100):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)

        self.embeddings = OpenAIEmbeddings()
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
    
        # prompt
        # 쿼리가 주어졌을 때, 쿼리에 대한 답변을 직접적으로하는 가설문서를 만들어냄 , 세부적이고 청크사이즈 지켜야한다. 
        # 질문에 직접적으로 답변할 수 있을 정도의 문서를 만드는 것이 목표 
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""Given the question '{query}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth.
            the document size has be exactly {chunk_size} characters.""",
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    # 가설문서 만드는 함수 
    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    # 만든 가설문서 임베딩하여 topk 반환하는 함수
    def retrieve(self, query, k=3):
        hypothetical_doc = self.generate_hypothetical_document(query)
        # 해당 가설문서와 유사한 것들을 벡터스토어에서 찾아낸다. => 아까 pdf에서 찾아낸다는 것 
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc


### Create a HyDe retriever instance

In [5]:
retriever = HyDERetriever(path)

### Demonstrate on a use case

In [6]:
test_query = "What is the main cause of climate change?"
results, hypothetical_doc = retriever.retrieve(test_query)

In [9]:
hypothetical_doc.split('\n')

['**Title: Understanding the Main Cause of Climate Change**',
 '',
 'Climate change primarily results from human activities, particularly the burning of fossil fuels such as coal, oil, and natural gas. This process releases significant amounts of carbon dioxide (CO2) and other greenhouse gases into the atmosphere. Deforestation exacerbates the issue by reducing the number of trees that can absorb CO2. Industrial processes, agriculture, and waste management also contribute to greenhouse gas emissions. The accumulation of these gases traps heat, leading to global warming and subsequent climate changes, impacting ecosystems and weather patterns worldwide.']

### Plot the hypothetical document and the retrieved documnets 

In [7]:
docs_content = [doc.page_content for doc in results]

print("hypothetical_doc:\n")
print(text_wrap(hypothetical_doc)+"\n")
show_context(docs_content)

hypothetical_doc:

**Title: Understanding the Main Cause of Climate Change**  Climate change primarily results from human activities,
particularly the burning of fossil fuels such as coal, oil, and natural gas. This process releases significant amounts
of carbon dioxide (CO2) and other greenhouse gases into the atmosphere. Deforestation exacerbates the issue by reducing
the number of trees that can absorb CO2. Industrial processes, agriculture, and waste management also contribute to
greenhouse gas emissions. The accumulation of these gases traps heat, leading to global warming and subsequent climate
changes, impacting ecosystems and weather patterns worldwide.

Context 1:
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmo

### 결론
- 질문을 기반으로 질문에 대한 직접적인 답변을 할 수 있을 정도의 가설문서를 만들어낸다.
- 해당 가설문서를 쿼리에 넣어 관련있는 문서를 찾아내는 것이 목표